In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()

env_variables_names = ["NEWS_API_KEY"]
for env_variable in env_variables_names:
    if env_variable not in os.environ:
        raise Exception(f"{env_variable} env variable should be defined")

module_path = os.path.abspath(os.path.join('..', '..', 'news-bias-agent'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext autoreload
%autoreload 2

In [2]:
from newsapi import NewsApiClient
import pandas as pd

newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

sources = newsapi.get_sources(language='en', country='us')
sources_df = pd.DataFrame(sources["sources"])
print(f"Available sources: {len(sources['sources'])}")
sources_df.info()
sources_df.head()


Available sources: 55
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           55 non-null     object
 1   name         55 non-null     object
 2   description  55 non-null     object
 3   url          55 non-null     object
 4   category     55 non-null     object
 5   language     55 non-null     object
 6   country      55 non-null     object
dtypes: object(7)
memory usage: 3.1+ KB


,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
1,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",https://www.aljazeera.com,general,en,us
2,ars-technica,Ars Technica,The PC enthusiast's resource. Power users and ...,https://arstechnica.com,technology,en,us
3,associated-press,Associated Press,The AP delivers in-depth coverage on the inter...,https://apnews.com/,general,en,us
4,axios,Axios,Axios are a new media company delivering vital...,https://www.axios.com,general,en,us


In [3]:
articles = newsapi.get_everything(
    q='climate change',
    sources='cnn,fox-news,abc-news',
    page_size=20,
    sort_by='relevancy' # Valid values: {"relevancy", "popularity", "publishedAt"}
)

for article in articles['articles']:
    print(f"{article['source']['name']}: {article['title']}")

CNN: New data raises questions about how much the Earth has warmed
Fox News: How future food domes could change the way you eat
CNN: Taps may run dry in this country, where the water crisis is so severe it can be seen from space
CNN: More redistricting bad news for Republicans: Texas may not net five GOP seats like they planned
ABC News: Lawsuit challenges the approval of an exploratory drilling program in Alaska
ABC News: WATCH: 'Mission possible' for sweets-loving twins
ABC News: WATCH: Typewriter artist creates his work one keystroke at a time
ABC News: WATCH: Train station in Ukraine pummeled by overnight Russian strikes
ABC News: WATCH: Millions of Americans brace for brutal cold and snowy weather
ABC News: WATCH: New audio reveals chaos in the cockpit of a terrifying mid-air scare in 2023
ABC News: WATCH: James Cameron talks 'Avatar: Fire and Ash'
ABC News: WATCH: Big cats rescued around the world find home in England
ABC News: WATCH: Officers use ‘grappler’ device to end police 

In [4]:
articles_df = pd.json_normalize(articles["articles"]) # this instead of pd.DataFrame(...) to handle nested dicts

In [5]:
articles_df

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Chris Mooney,New data raises questions about how much the E...,New climate data raises questions about how mu...,https://www.cnn.com/2025/12/15/climate/glosat-...,https://media.zenfs.com/en/cnn_articles_875/19...,2025-12-15T14:01:14Z,Chris Mooney is a Pulitzer Prize-winning journ...,cnn,CNN
1,None,How future food domes could change the way you...,Researchers from Osaka Metropolitan University...,https://www.foxnews.com/tech/how-future-food-d...,https://static.foxnews.com/foxnews.com/content...,2025-12-13T12:30:23Z,A futuristic food dome at Expo 2025 Osaka-Kans...,fox-news,Fox News
2,"Laura Paddison, CNN","Taps may run dry in this country, where the wa...",Tehran is grappling with a water crisis so sev...,https://www.cnn.com/2025/12/01/climate/iran-wa...,https://media.zenfs.com/en/cnn_articles_875/39...,2025-12-01T10:11:22Z,"Rows of worshippers, some with their faces rai...",cnn,CNN
3,"Edward Wu, Molly English, CNN",More redistricting bad news for Republicans: T...,The Republican plans to pick off five Democrat...,https://www.cnn.com/2025/12/14/politics/texas-...,https://media.zenfs.com/en/cnn_articles_875/7d...,2025-12-14T12:00:58Z,The Republican plans to pick off five Democrat...,cnn,CNN
4,BECKY BOHRER Associated Press,Lawsuit challenges the approval of an explorat...,A new lawsuit seeks to overturn the recent app...,https://abcnews.go.com/Business/wireStory/laws...,https://i.abcnewsfe.com/a/e8f20763-d370-41fe-8...,2025-12-12T00:07:22Z,"JUNEAU, Alaska -- Conservation groups and an I...",abc-news,ABC News
5,ABC News,WATCH: 'Mission possible' for sweets-loving twins,The 3-year-old twins worked together to get a ...,https://abcnews.go.com/GMA/Living/video/missio...,https://i.abcnewsfe.com/a/88ed5cd5-1d50-48b2-9...,2025-12-07T14:49:36Z,<ul><li>What to know about the case against Lu...,abc-news,ABC News
6,ABC News,WATCH: Typewriter artist creates his work one ...,Some artists paint with oil or acrylic. Others...,https://abcnews.go.com/International/video/typ...,https://i.abcnewsfe.com/a/1ffb9b54-b1de-4941-a...,2025-12-07T20:56:03Z,"<ul><li>Frigid temperatures in the Midwest, No...",abc-news,ABC News
7,ABC News,WATCH: Train station in Ukraine pummeled by ov...,Britt Clennett reports on the damage at a rail...,https://abcnews.go.com/International/video/tra...,https://i.abcnewsfe.com/a/1116d2d4-476a-467c-a...,2025-12-07T01:55:33Z,<ul><li>What to know about the case against Lu...,abc-news,ABC News
8,ABC News,WATCH: Millions of Americans brace for brutal ...,ABC News meteorologist Dani Beckstrom with the...,https://abcnews.go.com/world-news-tonight-with...,https://i.abcnewsfe.com/a/15754fd2-543d-403d-a...,2025-12-08T02:31:18Z,<ul><li>Defense Sec. Pete Hegseth defends stri...,abc-news,ABC News
9,ABC News,WATCH: New audio reveals chaos in the cockpit ...,Audio and video show when former Alaska Airlin...,https://abcnews.go.com/world-news-tonight-with...,https://i.abcnewsfe.com/a/bb5ae043-47d5-4b12-8...,2025-12-08T02:29:39Z,<ul><li>Defense Sec. Pete Hegseth defends stri...,abc-news,ABC News
